In [42]:
# requests 추가(통신을 위함)
!pip install langchain langchain_openai langgraph requests

# 현재 상황을 바탕으로 '할것, 먹을것' 추천해주는 챗봇

- 현재 상황 :
    - 나의 언급 : '심심해', '배고파' ...
    - 나의 위치
    - 현재 날씨
    - 지금 시간

- 추천 시 : 카카오맵 활용(실제 가게 주소 return)

In [53]:
import os
from google.colab import userdata

# LLM을 활용하기 위함
os.environ['OPENAI_API_KEY'] = userdata.get("sesac7_openAI2")

# 온도, 날씨 가져다 쓰기 위한 api
os.environ['WEATHER_API_KEY'] = userdata.get("weather_API")

# 카카오맵 api
os.environ['KAKAO_API_KEY'] = ''

In [44]:
from langchain_openai import ChatOpenAI
import json, requests, traceback      # traceback(로그 상세히/오류 디버깅)

# 시간 라이브러리
from datetime import datetime

# 랭그래프의 구조
from langgraph.graph import StateGraph, END     # 상태그래프와 END노드 세팅
from typing_extensions import TypedDict         # 상태그래프 커스터마이징

In [45]:
# # 추천에 필요한 요소들을 '상태'로 만듦
# class State(TypedDict):
#     user_input : str    # 나의 상태를 입력하는 str
#     location : str      # 나의 위치
#     timeslot : str      # 시간대(아침/점심/오후/저녁)
#     season : str        # 계절
#     weather : str       # 날씨
#     # 노드를 거치면서 결과물로 추가되는 상태
#     intent : str        # 의도(할 것 or 먹을 것)
#     recommended_items : list    # 추천하는 음식이나 활동 '리스트'
#     search_keyword : str        # kakao api에 특정 '장소'나 '가게'를 검색하기 위한 검색어 생성
#     recommended_place : dict    # 장소 추천 결과
#     final_message : str         # 현재 상황, 추천 결과를 고려하여서 사용자가 받을 최종 아웃풋

# # 전체 구조
# # 워크플로우에 포함될 상태 그래프
# workflow = StateGraph(State)

# # 노드 추가
# workflow.add_node('classify_intent', classify_intent)   # 사용자의 입력 문장을 보고 할것 or 먹을것 분류
# workflow.add_node('get_time', get_time)
# workflow.add_node('get_season', get_season)
# workflow.add_node('get_weather', get_weather)

# # 할것 추천 노드, 먹을 것 추천 노드
# workflow.add_node('recommend_food', recommend_food)
# workflow.add_node('recommend_activity', recommend_activity)

# # 구체적인 장소를 검색하기 위한 '검색어 생성'
# workflow.add_node('generate_keyword', generate_keyword)

# # 카카오맵 api에 장소 검색
# workflow.add_node('search_place', search_place)

# # 요약해서 친절하게 답변 생성
# workflow.add_node('summarize_output', summarize_output)

# # 파악할 수 없는 의도를 예외처리
# workflow.add_node('handle_unsupported', handle_unsupported)

## 노드에서 '에이전트'와 '툴'을 분리하기

In [46]:
# 2가지 타입의 llm을 정의, 같은 에이전트지만 'return 받는 값'에 따라 쪼개줌
llm = ChatOpenAI()

llm_json = ChatOpenAI(
    model = 'gpt-4o-mini',
    api_key = os.environ['OPENAI_API_KEY'],
    temperature= 0.5,
    model_kwargs={'response_format':{'type':'json_object'}}
) # 추천받은 결과가 여러 개일 때, json 포맷으로 들어오게 하기 위함

llm_normal = ChatOpenAI(
    model = 'gpt-4o-mini',
    api_key = os.environ['OPENAI_API_KEY'],
    temperature= 0.5
)

In [47]:
# 사용자 입력의 의도를 분류 : 할것, 먹을것, 상관없는것(언노운)
# 원래라면 매개변수, 리턴값 == StateGraph의 형태로 지정되어야 하는데, 코드로 변환
def classify_intent(state:dict) -> dict:
    # 사용자의 입력 내용을 가져옴
    # 딕셔너리.get('키', 'B') -> '키'에 해당하는 값이 없으면 'B'를 가져와라
    user_input = state.get('user_input', '')

    prompt = f"""
      당신은 사용자의 자연어 입력을 food / activity / unknown 중 하나로 분류하는 AI입니다.

      입력: "{user_input}"

      분류 기준:
      - 음식 관련 표현 → "food" (예: 배고파, 뭐 먹지, 야식 추천해줘 등)
      - 활동 관련 표현 → "activity" (예: 심심해, 뭐 하지, 놀고 싶어 등)
      - 증상, 감정, 질문, 애매한 표현 → "unknown"

      조금 애매한 표현이라도 의미가 보이면 food 또는 activity로 분류하세요.

      출력은 반드시 다음 중 하나의 JSON 배열 또는 객체로 작성하세요:
      - 배열: ["food"]
      - 객체: {{ "intent": ["food"] }}
    """

    response = llm_json.invoke([{'role':'user', 'content':prompt.strip()}])

    # 정답에 strip()해서 양쪽 공백 제거
    intent = response.content.strip()
    # classify_intent({'user_input':'내가 좋아하는 색깔은 파란색이야'})
    # print(intent)

    try:
        # intent(str)을 json.loads()로 적용 -> 타입을 변경 / 결과를 분리
        parsing = json.loads(intent)

        # 1. isinstance(A, B) -> A가 B타입이냐?
        # 2. parsing이 존재?
        # 3. parsing[0] 존재?
        if isinstance(parsing, list) and parsing and parsing[0]:
            # **state "딕셔너리 언패킹"
            # 'intent'(키) : parsing[0](값)
            return {**state, 'intent':parsing[0]}

        if isinstance(parsing, dict) :
            if 'intent' in parsing:
                value = parsing['intent']

                # 1. value가 list야?
                # 2. value가 존재해?
                # 3. value의 [0]번째 값이 food나 activity야?
                if isinstance(value, list) and value and value[0] in ['food', 'activity']:
                    return {**state, 'intent':value[0]}

                for key in ['food', 'activity']:
                    if key in parsing:
                        return {**state, 'intent':key}

    except Exception as e:
        print("에러발생 :")
        print(traceback.format_exc())   # traceback 라이브러리로 보다 자세한 오류 로그 셋팅

    return {**state, 'intent':'unknown'}

statement = classify_intent({'user_input':'몬헌 음식'})

In [48]:
statement

{'user_input': '몬헌 음식', 'intent': 'food'}

In [49]:
from zoneinfo import ZoneInfo
# 툴(역할)
# state를 받아서 언패킹해서 timeslot을 return
def get_time(state:dict) -> dict:
    # 함수가 실행된 순간의 시(hour), 00~24시로 들어옴
    KST = ZoneInfo('Asia/Seoul')
    hour = datetime.now(KST).hour
    # hour = datetime.now().hour

    if 5 <= hour < 11:
        return {**state, 'timeslot':'아침'}

    elif 11 <= hour <14:
        return {**state, 'timeslot':'점심'}

    elif 14 <= hour <18:
        return {**state, 'timeslot':'오후'}

    else:
        return {**state, 'timeslot':'저녁'}

In [50]:
# 툴(역할)
# state를 받아서 언패킹해서 timeslot을 return
def get_season(state:dict) -> dict:
    # 함수가 실행된 순간의 시(hour), 00~24시로 들어옴
    KST = ZoneInfo('Asia/Seoul')
    m = datetime.now(KST).month

    if 3 <= m < 6:
        return {**state, 'timeslot':'봄'}

    elif 6 <= m < 10:
        return {**state, 'timeslot':'여름'}

    elif 10 <= m < 11:
        return {**state, 'timeslot':'가을'}

    else:
        return {**state, 'timeslot':'겨울'}

In [56]:
def get_weather(state:dict) -> dict:

    #특정 인터넷 주소에 신호를 보내고 -> 받아오는
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        'q' : 'Seoul',
        'appid' : os.environ['WEATHER_API_KEY'],
        'lang' : 'kr',
        'units' : 'metric'
    }
    #requests.get(주소, 요청사항) : 특정한 주소에, 필요한 요청사항을 담아 보냄
    response = requests.get(url, params=params)

    #<200> -> 성공 <400>->오류
    #응답 코드가 '실패'인 경우->코드 확인
    response.raise_for_status()

    #오류 없음 -> 데이터에서 '날씨'를 의미하는 부분 추가
    weather_data = response.json()
    weather = weather_data['weather'][0]['main']

    print(weather_data)
    return {**state, 'weather' : weather}

get_weather({})

{'coord': {'lon': 126.9778, 'lat': 37.5683}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': '구름조금', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 31.76, 'feels_like': 38.76, 'temp_min': 31.76, 'temp_max': 33.78, 'pressure': 1008, 'humidity': 70, 'sea_level': 1008, 'grnd_level': 999}, 'visibility': 10000, 'wind': {'speed': 7.72, 'deg': 200}, 'clouds': {'all': 40}, 'dt': 1756108322, 'sys': {'type': 1, 'id': 8105, 'country': 'KR', 'sunrise': 1756068981, 'sunset': 1756116757}, 'timezone': 32400, 'id': 1835848, 'name': 'Seoul', 'cod': 200}


{'weather': 'Clouds'}